In [1]:
#====================================
# Author: gmd8si
# Date: 16, November, 2016
#====================================
%matplotlib inline
import tensorflow as tf
import numpy as np
import pandas as pd

from detector_dr import Detector
from util import load_image           # Questionalable since we can use our own loading pipeline
import os
import matplotlib.pyplot as plt

#================================================
# Setting Hyper-Parameters 
#================================================

#weight_path = '../data/caffe_layers_value.pickle'
#model_path = '../models/caltech256/'
PRETRAINED_MODEL_PATH= None #'../models/caltech256/model-0'
N_EPOCHS = 10000
INIT_LEARNING_RATE = 0.01
WEIGHT_DECAY_RATE = 0.0005
MOMENTUM = 0.9
IMAGE_HEIGHT  = 512    #960
IMAGE_WIDTH   = 512    #720
NUM_CHANNELS  = 3
BATCH_SIZE = 50
N_LABELS = 5
DROPOUT = 0.50  
LOGS_PATH = './tensorflow_logs/new'
WEIGHT_PATH = None
TRAINSET_PATH = '/home/gmd8si/Weakly_detector/data/kaggle512_train1000.csv'
TESTSET_PATH ='/home/gmd8si/Weakly_detector/data/kaggle512_test.csv'

# Reading Data

In [2]:
#=======================================================================================================
# Reading Training data from CSV FILE
#=======================================================================================================
# load csv content, reading labels and filepaths
#csv_path = tf.train.string_input_producer(['/home/gmd8si/Weakly_detector/data/kaggle512_train.csv'])
#def load_data(data_path, batch_size):
csv_path = tf.train.string_input_producer([TRAINSET_PATH])
textReader = tf.TextLineReader()
_, csv_content = textReader.read(csv_path)
im_name, im_label = tf.decode_csv(csv_content, record_defaults=[[""], [1]])



# load images, and convert labels into one_hot encoded form
# DO WE NEED HOT-ENCODED VECTORS

im_content = tf.read_file(im_name)
image = tf.image.decode_jpeg(im_content, channels=3)
image = tf.cast(image, tf.float32) / 255. # could be unnecessary
#image = image.set_shape([IMAGE_HEIGHT, IMAGE_WIDTH, NUM_CHANNELS])
size = tf.cast([512, 512], tf.int32)
image = tf.image.resize_images(image, size)
#label = tf.one_hot(label, 5, 1, 0 )
label = tf.cast(im_label, tf.int32) # unnecessary
# Make Batches of images with shuffling
min_after_dequeue = 60                                               # Defines how big a buffer we will randomly sample from -- bigger means better shuffling but slower start up and more memory used
capacity = min_after_dequeue + 3 * BATCH_SIZE                         # Capacity must be larger than min_after_dequeue and the amount larger determines the maximum we will prefetch.
train_image_batch, train_label_batch = tf.train.batch(
    [image, label], batch_size=BATCH_SIZE)#, capacity=capacity,min_after_dequeue=min_after_dequeue) # shuffle_batch , capacity=capacity,min_after_dequeue=10



In [26]:
#=======================================================================================================
# Reading Training data from CSV FILE
#=======================================================================================================
# load csv content, reading labels and filepaths
#csv_path = tf.train.string_input_producer(['/home/gmd8si/Weakly_detector/data/kaggle512_train.csv'])
#def load_data(data_path, batch_size):
csv_path = tf.train.string_input_producer([TRAINSET_PATH], num_epochs=10000, shuffle=True, capacity = 60)
textReader = tf.TextLineReader()
_, csv_content = textReader.read(csv_path)
im_name, im_label = tf.decode_csv(csv_content, record_defaults=[[""], [1]])

# load images, and convert labels into one_hot encoded form
# DO WE NEED HOT-ENCODED VECTORS

im_content = tf.read_file(im_name)
image = tf.image.decode_jpeg(im_content, channels=3)
image = tf.cast(image, tf.float32) / 255. # could be unnecessary
#image = image.set_shape([IMAGE_HEIGHT, IMAGE_WIDTH, NUM_CHANNELS])
size = tf.cast([512, 512], tf.int32)
image = tf.image.resize_images(image, size)
#label = tf.one_hot(label, 5, 1, 0 )
label = tf.cast(im_label, tf.int32) # unnecessary
# Make Batches of images with shuffling
min_after_dequeue = 60                                               # Defines how big a buffer we will randomly sample from -- bigger means better shuffling but slower start up and more memory used
capacity = min_after_dequeue + 3 * BATCH_SIZE                         # Capacity must be larger than min_after_dequeue and the amount larger determines the maximum we will prefetch.
train_image_batch, train_label_batch = tf.train.batch([image, label], batch_size=BATCH_SIZE)

In [27]:
with tf.Session() as sess:
    # Initializing all variables is necessary as for num_epoches, the ftn string_input_producer creates an internal
    # variable to track the current epoch index, and therefore it needs to be initialized before.
    sess.run(tf.initialize_all_variables())    
    sess.run(tf.initialize_local_variables())
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    try: 
        #while not coord.should_stop():
        for i in range (25):
            
            #print(sess.run(im_name))
            print i+1
            s = sess.run(train_image_batch)
            print s.shape
    except tf.errors.OutOfRangeError:
        print ('Epoch Reached, Done Training')
    

1
(50, 512, 512, 3)
2
(50, 512, 512, 3)
3
(50, 512, 512, 3)
4
(50, 512, 512, 3)
5
(50, 512, 512, 3)
6
(50, 512, 512, 3)
7
(50, 512, 512, 3)
8
(50, 512, 512, 3)
9
(50, 512, 512, 3)
10
(50, 512, 512, 3)
11
(50, 512, 512, 3)
12
(50, 512, 512, 3)
13
(50, 512, 512, 3)
14
(50, 512, 512, 3)
15
(50, 512, 512, 3)
16
(50, 512, 512, 3)
17
(50, 512, 512, 3)
18
(50, 512, 512, 3)
19
(50, 512, 512, 3)
20
(50, 512, 512, 3)
21
Epoch Reached, Done Training


In [17]:
import csv

TRAINSET_PATH = '/home/gmd8si/Weakly_detector/data/kaggle512_train1000.csv'
csvfile = open(TRAINSET_PATH, "rb")
reader = csv.reader(csvfile)
#row_count = sum(1 for row in reader)
#print row_count

#fnames, labels = [], []  
for row in reader:
    fnames.append(row)
#    labels.append(int(row[1]))
print len(fnames)
csv_path = tf.train.string_input_producer([fnames])
del fnames[:]

#csvfile.close()
    
    #if num_labels == 0:
        #return fnames
    #else:
        #return fnames, labels
    #return fnames, labels
#print len(fnames)
'''

fileO = csv.reader(TRAINSET_PATH)
data = list(fileO)
rc = len(data)
#row_count = sum(1 for row in fileO)
print rc

for c in enumerate(fileO,0):
    print c

'''

1002


ValueError: Shapes (1002, 2) and () are not compatible

In [16]:
with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    count = 0
    
    try:
        for i in range (100):
            count = count + 1
            print count
            batch_x, batch_y = sess.run([train_image_batch, train_label_batch])
            print batch_y.shape
    #print a.shape
    #la = sess.run(label, feed_dict={data_path:TRAINSET_PATH})  
    except tf.errors.OutOfRangeError:
        print ('Done Training -- epoch Limit reached')
   
    '''
    try:
        #while not coord.should_stop():
        for i in range(100):
            la = sess.run(train_label_batch, feed_dict={data_path:TRAINSET_PATH})  
            print la.shape
    except tf.errors.OutOfRangeError:
        print ('Done Training -- epoch Limit reached')
    finally:
        coord.request_stop()
        #lt.imshow(im)
    '''

1
(50,)
2
(50,)
3
(50,)
4
(50,)
5
(50,)
6
(50,)
7
(50,)
8
(50,)
9
(50,)
10
(50,)
11
(50,)
12
(50,)
13
(50,)
14
(50,)
15
(50,)
16
(50,)
17
(50,)
18
(50,)
19
(50,)
20
(50,)
21


KeyboardInterrupt: 

In [6]:
print batch_y.shape

(30,)


# Placeholders

In [3]:
learning_rate = tf.placeholder( tf.float32, [])
images_tf = tf.placeholder( tf.float32, [None, 512, 512, 3], name="images")
labels_tf = tf.placeholder( tf.int32, [None], name='labels')  # the dimensions could be [None,N_CLASSES]

In [4]:
detector = Detector( N_LABELS)

In [5]:
p1,p2,p3,p4,conv5, conv6, gap, output = detector.inference(images_tf)
loss_tf = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits( output, labels_tf ))

Below is something I dont reckon

In [6]:
weights_only = filter( lambda x: x.name.endswith('W:0'), tf.trainable_variables() )
weight_decay = tf.reduce_sum(tf.pack([tf.nn.l2_loss(x) for x in weights_only])) * WEIGHT_DECAY_RATE
loss_tf += weight_decay

In [7]:
# Saver Operation to save and restore all variables, first create directory
ckpt_dir = "./ckpt_dir"
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)
saver = tf.train.Saver(max_to_keep=50)   #Indicates max number of recent checkpoint files to keep

In [8]:
sess = tf.InteractiveSession()
optimizer = tf.train.MomentumOptimizer( learning_rate, MOMENTUM)
grads_and_vars = optimizer.compute_gradients( loss_tf )
grads_and_vars = map(lambda gv: (gv[0], gv[1]) if ('conv6' in gv[1].name or 'GAP' in gv[1].name) else (gv[0]*0.1, gv[1]), grads_and_vars)
#grads_and_vars = [(tf.clip_by_value(gv[0], -5., 5.), gv[1]) for gv in grads_and_vars]
train_op = optimizer.apply_gradients( grads_and_vars )

In [9]:
tf.initialize_all_variables().run()

In [10]:
if PRETRAINED_MODEL_PATH:
    print "Pretrained"
    saver.restore(sess, PRETRAINED_MODEL_PATH)

In [11]:
# They use this testset index, no idea what is it for
#testset.index  = range( len(testset) )

In [16]:
if not os.path.exists('./results'):
    os.makedirs('./results')
f_log = open('./results/DR_1_vggCAM.txt', 'w')

iterations = 0
loss_list = []

In [21]:
for start, end in zip(range(0, 660, 60), range(60, 660, 60)):
    print start, end
#print zip(range(0, 660, 60), range(60, 660, 60))

0 60
60 120
120 180
180 240
240 300
300 360
360 420
420 480
480 540
540 600


In [ ]:


for epoch in range(N_EPOCHS):
    
    #Simply setting trainset size
    #trainset.index = range(len(trainset))
    #trainset = trainset.ix[ np.random.permutation( len(trainset) )]

    for start, end in zip(range( 0, len(trainset)+batch_size, batch_size), 
                          range(batch_size, len(trainset)+batch_size, batch_size)):

        #current_data = trainset[start:end]                           # for 1st iteration it is trainset[0:60], so like defining 1st batch of batch_size
        #current_image_paths = current_data['image_path'].values      # loading images from 1st batch 
        #current_images = np.array(map(lambda x: load_image(x), current_image_paths)) # loading images from 1st batch

        #good_index = np.array(map(lambda x: x is not None, current_images)) #indexes of images

        #current_data = current_data[good_index]  
        #current_images = np.stack(current_images[good_index])    # Making Batch of images
        #current_labels = current_data['label'].values            # Making batch of labels

        _, loss_val, output_val = sess.run(
                [train_op, loss_tf, output],
                feed_dict={
                    learning_rate: init_learning_rate,
                    images_tf: current_images,
                    labels_tf: current_labels
                    })

        loss_list.append(loss_val)                              # This must be for visualizing loss curve 

        iterations += 1
        if iterations % 5 == 0:
            print "======================================"
            print "Epoch", epoch, "Iteration", iterations
            print "Processed", start, '/', len(trainset)

            label_predictions = output_val.argmax(axis=1)
            acc = (label_predictions == current_labels).sum()

            print "Accuracy:", acc, '/', len(current_labels)
            print "Accuracy Percentage:", (acc/len(current_labels))*100
            print "Training Loss:", np.mean(loss_list)
            print "\n"
            loss_list = []

    n_correct = 0
    n_data = 0
    for start, end in zip(range(0, len(testset)+batch_size, batch_size),
                          range(batch_size, len(testset)+batch_size, batch_size)
            ):
        #current_data = testset[start:end]
        #current_image_paths = current_data['image_path'].values
        #current_images = np.array(map(lambda x: load_image(x), current_image_paths))

        #good_index = np.array(map(lambda x: x is not None, current_images))

        #current_data = current_data[good_index]
        #current_images = np.stack(current_images[good_index])
        #current_labels = current_data['label'].values

        output_vals = sess.run(output, lfeed_dict={images_tf:current_images})

        label_predictions = output_vals.argmax(axis=1)
        acc = (label_predictions == current_labels).sum()

        n_correct += acc
        n_data += len(current_data)

    acc_all = n_correct / float(n_data)
    f_log.write('epoch:'+str(epoch)+'\tacc:'+str(acc_all) + '\n')
    print "$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$"
    print 'epoch:'+str(epoch)+'\tacc:'+str(acc_all) + '\n'
    print "$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$"

    saver.save( sess, os.path.join( model_path, 'model'), global_step=epoch)

    init_learning_rate *= 0.99